In [1]:
from warp.jax_experimental.ffi import jax_callable, jax_kernel 
import jax.numpy as jnp
import jax
import warp as wp

@wp.kernel
def scale_kernel(a: wp.array(dtype=float),
                 s: wp.array(dtype=float),
                 output: wp.array(dtype=float)):
    tid = wp.tid()
    output[tid] = a[tid] * s[0]


jax_scale = jax_kernel(scale_kernel)

@jax.jit
def f(a, s):
    return jax_scale(a, s)  # ERROR: traced scalar argument

a = jnp.arange(10, dtype=jnp.float32)
s = jnp.array([2.0])

print(f(a, s))

Warp 1.7.0.dev20250223 initialized:
   CUDA Toolkit 12.8, Driver 12.2
   Devices:
     "cpu"      : "x86_64"
     "cuda:0"   : "NVIDIA A40" (44 GiB, sm_86, mempool enabled)
     "cuda:1"   : "NVIDIA A40" (47 GiB, sm_86, mempool enabled)
     "cuda:2"   : "NVIDIA A40" (47 GiB, sm_86, mempool enabled)
     "cuda:3"   : "NVIDIA A40" (47 GiB, sm_86, mempool enabled)
     "cuda:4"   : "NVIDIA A40" (47 GiB, sm_86, mempool enabled)
     "cuda:5"   : "NVIDIA A40" (47 GiB, sm_86, mempool enabled)
     "cuda:6"   : "NVIDIA A40" (47 GiB, sm_86, mempool enabled)
     "cuda:7"   : "NVIDIA A40" (47 GiB, sm_86, mempool enabled)
   CUDA peer access:
     Supported fully (all-directional)
   Kernel cache:
     /home/haw027/.cache/warp/1.7.0.dev20250223
Module __main__ 6566af2 load on device 'cuda:0' took 239.33 ms  (compiled)
[Array([ 0.,  2.,  4.,  6.,  8., 10., 12., 14., 16., 18.], dtype=float32)]


In [1]:
from warp.jax_experimental.ffi import jax_callable, jax_kernel 
import jax.numpy as jnp
import jax
import warp as wp

@wp.kernel
def scale_kernel(a: wp.array(dtype=float),
                 s: float,  # scalar input
                 output: wp.array(dtype=float)):
    tid = wp.tid()
    output[tid] = a[tid] * s


jax_scale = jax_kernel(scale_kernel)

@jax.jit
def f():
    a = jnp.arange(10, dtype=jnp.float32)
    return jax_scale(a, 2.0)  # ok: constant scalar argument

print(f())

Warp 1.7.0.dev20250223 initialized:
   CUDA Toolkit 12.8, Driver 12.2
   Devices:
     "cpu"      : "x86_64"
     "cuda:0"   : "NVIDIA A40" (44 GiB, sm_86, mempool enabled)
     "cuda:1"   : "NVIDIA A40" (47 GiB, sm_86, mempool enabled)
     "cuda:2"   : "NVIDIA A40" (47 GiB, sm_86, mempool enabled)
     "cuda:3"   : "NVIDIA A40" (47 GiB, sm_86, mempool enabled)
     "cuda:4"   : "NVIDIA A40" (47 GiB, sm_86, mempool enabled)
     "cuda:5"   : "NVIDIA A40" (47 GiB, sm_86, mempool enabled)
     "cuda:6"   : "NVIDIA A40" (47 GiB, sm_86, mempool enabled)
     "cuda:7"   : "NVIDIA A40" (47 GiB, sm_86, mempool enabled)
   CUDA peer access:
     Supported fully (all-directional)
   Kernel cache:
     /home/haw027/.cache/warp/1.7.0.dev20250223
Module __main__ 7d593f8 load on device 'cuda:0' took 0.38 ms  (cached)
[Array([ 0.,  2.,  4.,  6.,  8., 10., 12., 14., 16., 18.], dtype=float32)]


In [2]:
@wp.kernel
def scale_kernel(a: wp.array(dtype=float), s: float, output: wp.array(dtype=float)):
    tid = wp.tid()
    output[tid] = a[tid] * s

@wp.kernel
def scale_vec_kernel(a: wp.array(dtype=wp.vec2), s: float, output: wp.array(dtype=wp.vec2)):
    tid = wp.tid()
    output[tid] = a[tid] * s


# The Python function to call.
# Note the argument type annotations, just like Warp kernels.
def example_func(
    # inputs
    a: wp.array(dtype=float),
    b: wp.array(dtype=wp.vec2),
    s: float,
    # outputs
    c: wp.array(dtype=float),
    d: wp.array(dtype=wp.vec2),
):
    # launch multiple kernels
    wp.launch(scale_kernel, dim=a.shape, inputs=[a, s], outputs=[c])
    wp.launch(scale_vec_kernel, dim=b.shape, inputs=[b, s], outputs=[d])


jax_func = jax_callable(example_func, num_outputs=2)

@jax.jit
def f():
    # inputs
    a = jnp.arange(10, dtype=jnp.float32)
    b = jnp.arange(10, dtype=jnp.float32).reshape((5, 2))  # wp.vec2
    s = 2.0

    print("a: ", a)
    print("s: ", s)
    # output shapes
    output_dims = {"c": a.shape, "d": b.shape}

    c, d = jax_func(a, b, s, output_dims=output_dims)

    return c, d

r1, r2 = f()
print(r1)
print(r2)

a:  Traced<ShapedArray(float32[10])>with<DynamicJaxprTrace>
s:  2.0
Module __main__ 65c51f1 load on device 'cuda:0' took 337.48 ms  (compiled)
[ 0.  2.  4.  6.  8. 10. 12. 14. 16. 18.]
[[ 0.  2.]
 [ 4.  6.]
 [ 8. 10.]
 [12. 14.]
 [16. 18.]]


In [ ]:
@wp.kernel
def scale_kernel(a: wp.array(dtype=float), s: wp.array(dtype=float), output: wp.array(dtype=float)):
    tid = wp.tid()
    output[tid] = a[tid] * s[0]

@wp.kernel
def scale_vec_kernel(a: wp.array(dtype=wp.vec2), s: wp.array(dtype=float), output: wp.array(dtype=wp.vec2)):
    tid = wp.tid()
    output[tid] = a[tid] * s[0]


# The Python function to call.
# Note the argument type annotations, just like Warp kernels.
def example_func(
    # inputs
    a: wp.array(dtype=float),
    b: wp.array(dtype=wp.vec2),
    s: wp.array(dtype=float),
    # outputs
    c: wp.array(dtype=float),
    d: wp.array(dtype=wp.vec2),
):
    # launch multiple kernels
    wp.launch(scale_kernel, dim=a.shape, inputs=[a, s], outputs=[c])
    wp.launch(scale_vec_kernel, dim=b.shape, inputs=[b, s], outputs=[d])


jax_func = jax_callable(example_func, num_outputs=2)

@jax.jit
def f():
    # inputs
    a = jnp.arange(10, dtype=jnp.float32)
    b = jnp.arange(10, dtype=jnp.float32).reshape((5, 2))  # wp.vec2
    s = jnp.array([2.0])
    print(s)
    print(s[0])

    # output shapes
    output_dims = {"c": a.shape, "d": b.shape}

    c, d = jax_func(a, b, s, output_dims=output_dims)

    return c, d

r1, r2 = f()
print(r1)
print(r2)

Traced<ShapedArray(float32[1])>with<DynamicJaxprTrace>
Traced<ShapedArray(float32[])>with<DynamicJaxprTrace>
a:  Traced<ShapedArray(float32[10])>with<DynamicJaxprTrace>
s:  Traced<ShapedArray(float32[1])>with<DynamicJaxprTrace>
Module __main__ 2f5dbcd load on device 'cuda:0' took 324.24 ms  (compiled)
[ 0.  2.  4.  6.  8. 10. 12. 14. 16. 18.]
[[ 0.  2.]
 [ 4.  6.]
 [ 8. 10.]
 [12. 14.]
 [16. 18.]]


In [1]:
import json
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
from os.path import join
import trimesh
from b3d.chisight.gen3d.dataloading import (
    get_initial_state,
    load_trial,
)
import b3d.chisight.gen3d.settings as settings
# import jax.numpy as jnp
# import jax


scenario = 'drop'
trial_name = 'pilot_it2_drop_all_bowls_box_0002'

hdf5_file_path = "/ccn2/u/rmvenkat/data/testing_physion/regenerate_from_old_commit/test_humans_consolidated/lf_0"
mesh_file_path = "/ccn2/u/rmvenkat/data/all_flex_meshes/"
pred_file_path = "/ccn2/u/haw027/b3d_ipe/pred_files/gt_info/gt_correct.json"

with open(pred_file_path) as f:
    pred_file_all = json.load(f)
pred_file = pred_file_all[trial_name]

all_meshes = {}
for path, dirs, files in os.walk(mesh_file_path):
    for name in files + dirs:
        if name.endswith(".obj"):
            mesh = trimesh.load(os.path.join(path, name))
            all_meshes[name[:-4]] = mesh


hdf5_file_path = join(
    hdf5_file_path,
    scenario + "_all_movies",
    f"{trial_name}.hdf5",
)

(
    rgbds_original,
    seg_arr_original,
    object_ids,
    object_segmentation_colors,
    background_areas,
    camera_pose,
) = load_trial(hdf5_file_path, 15)

hyperparams = settings.hyperparams

initial_state, hyperparams = get_initial_state(
    pred_file,
    object_ids,
    object_segmentation_colors,
    all_meshes,
    seg_arr_original[0],
    rgbds_original[0],
    hyperparams,
)

/ccn2/u/haw027/miniconda3/envs/genjax/lib/python3.11/site-packages/genjax/_src/generative_functions/distributions/distribution.py:434: UserWarning: You should supply a name argument to exact_density
  warnings.warn("You should supply a name argument to exact_density")
/ccn2/u/haw027/miniconda3/envs/genjax/lib/python3.11/site-packages/genjax/_src/generative_functions/distributions/distribution.py:434: UserWarning: You should supply a name argument to exact_density
  warnings.warn("You should supply a name argument to exact_density")


Warp 1.7.0.dev20250223 initialized:
   CUDA Toolkit 12.8, Driver 12.2
   Devices:
     "cpu"      : "x86_64"
     "cuda:0"   : "NVIDIA A40" (44 GiB, sm_86, mempool enabled)
   Kernel cache:
     /home/haw027/.cache/warp/1.7.0.dev20250223
Module warp.sim.inertia 185ee85 load on device 'cuda:0' took 0.30 ms  (cached)
Module warp.sim.collide 26674e6 load on device 'cuda:0' took 1.59 ms  (cached)


In [2]:
hyperparams

{'pose_kernel': GaussianVMFPoseDriftKernel(std=0.02, concentration=1000.0),
 'mu': 0.25,
 'restitution': 0.4,
 'fps': 100,
 'sim_substeps': 10,
 'g': -9.80665,
 'sim_dt': 0.001,
 'meshes': {1: Mesh(vertices=[[ 0.333334    0.          0.333334  ]
   [ 0.333334    0.         -1.        ]
   [ 0.333334    0.          1.        ]
   [ 0.333334    0.         -0.333334  ]
   [ 0.333334    0.01        0.333334  ]
   [ 0.333334    0.01       -1.        ]
   [ 0.333334    0.01        1.        ]
   [ 0.333334    0.01       -0.333334  ]
   [ 0.333334    0.00333333 -1.        ]
   [ 0.333334    0.00333333  1.        ]
   [ 0.333334    0.00666667 -1.        ]
   [ 0.333334    0.00666667  1.        ]
   [-1.          0.          0.333334  ]
   [-1.          0.         -1.        ]
   [-1.          0.          1.        ]
   [-1.          0.         -0.333334  ]
   [-1.          0.01        0.333334  ]
   [-1.          0.01       -1.        ]
   [-1.          0.01        1.        ]
   [-1.         

In [3]:
initial_state["prev_model"]._geo_type

Array([5, 5, 5, 7], dtype=int32)

In [4]:
from b3d.physics.physics_utils import step

In [5]:
model_new, state_new = step(initial_state["prev_model"], initial_state["prev_state"], hyperparams)

Module b3d.physics.physics_utils ad9cc19 load on device 'cuda:0' took 0.46 ms  (cached)


In [6]:
# rigid_contact_count, rigid_contact_broad_shape0, rigid_contact_broad_shape1, rigid_contact_point_id, rigid_contact_shape0, rigid_contact_shape1, rigid_contact_point0, rigid_contact_point1, rigid_contact_offset0, rigid_contact_offset1, rigid_contact_normal, rigid_contact_thickness, rigid_contact_tids = step(initial_state["prev_model"], initial_state["prev_state"], hyperparams["rigid_contact_max"], hyperparams["shape_contact_pair_count"], hyperparams["shape_ground_contact_pair_count"], hyperparams["rigid_contact_margin"], hyperparams["ground"], hyperparams["body_count"], hyperparams["gravity"], hyperparams["sim_dt"])

In [7]:
# stepped_model, stepped_state = step(initial_state["prev_model"], initial_state["prev_state"], hyperparams["rigid_contact_max"], hyperparams["shape_contact_pair_count"], hyperparams["shape_ground_contact_pair_count"], hyperparams["rigid_contact_margin"], hyperparams["ground"], hyperparams["body_count"], hyperparams["gravity"], hyperparams["sim_dt"])

In [8]:
for i in range(250):
    print(f"step: {i}")
    stepped_model, stepped_state = step(initial_state["prev_model"], initial_state["prev_state"], hyperparams)
    initial_state["prev_model"] = stepped_model
    initial_state["prev_state"] = stepped_state

step: 0
step: 1
step: 2
step: 3
step: 4
step: 5
step: 6
step: 7
step: 8
step: 9
step: 10
step: 11
step: 12
step: 13
step: 14
step: 15
step: 16
step: 17
step: 18
step: 19
step: 20
step: 21
step: 22
step: 23
step: 24
step: 25
step: 26
step: 27
step: 28
step: 29
step: 30
step: 31
step: 32
step: 33
step: 34
step: 35
step: 36
step: 37
step: 38
step: 39
step: 40
step: 41
step: 42
step: 43
step: 44
step: 45
step: 46
step: 47
step: 48
step: 49
step: 50
step: 51
step: 52
step: 53
step: 54
step: 55
step: 56
step: 57
step: 58
step: 59
step: 60
step: 61
step: 62
step: 63
step: 64
step: 65
step: 66
step: 67
step: 68
step: 69
step: 70
step: 71
step: 72
step: 73
step: 74
step: 75
step: 76
step: 77
step: 78
step: 79
step: 80
step: 81
step: 82
step: 83
step: 84
step: 85
step: 86
step: 87
step: 88
step: 89
step: 90
step: 91
step: 92
step: 93
step: 94
step: 95
step: 96
step: 97
step: 98
step: 99
step: 100
step: 101
step: 102
step: 103
step: 104
step: 105
step: 106
step: 107
step: 108
step: 109
step: 110


In [4]:
for i in range(250):
    print(i)
    # initial_state["prev_state"].clear_forces()
    # initial_state["prev_model"].clear_old_count()
    stepped_model, stepped_state = step(initial_state["prev_model"], initial_state["prev_state"], hyperparams)
    initial_state["prev_model"] = stepped_model
    initial_state["prev_state"] = stepped_state

0
Module b3d.physics.physics_utils b55f565 load on device 'cuda:0' took 0.40 ms  (cached)
1
2
3


E0225 08:33:07.985645 1263678 pjrt_stream_executor_client.cc:3045] Execution of replica 0 failed: INTERNAL: CUDA error: Failed to launch CUDA kernel: wrapped_broadcast; block dims: 18x1x1; grid dims: 1x1x1; shared memory size: 0: CUDA_ERROR_ILLEGAL_ADDRESS: an illegal memory access was encountered


ValueError: INTERNAL: CUDA error: Failed to launch CUDA kernel: wrapped_broadcast; block dims: 18x1x1; grid dims: 1x1x1; shared memory size: 0: CUDA_ERROR_ILLEGAL_ADDRESS: an illegal memory access was encountered

In [ ]:
for i in range(250):
    print(i)
    stepped_model, stepped_state = step(initial_state["prev_model"], initial_state["prev_state"], hyperparams)
    initial_state["prev_model"] = stepped_model
    initial_state["prev_state"] = stepped_state

0
Module b3d.physics.physics_utils b55f565 load on device 'cuda:0' took 0.43 ms  (cached)
1
2
Traceback (most recent call last):
  File "/ccn2/u/haw027/miniconda3/envs/genjax/lib/python3.11/site-packages/warp/jax_experimental/ffi.py", line 547, in ffi_callback
    self.func(*arg_list)
  File "/home/haw027/code/b3d/src/b3d/physics/physics_utils.py", line 674, in simulate
    dim=rigid_contact_max.list()[0].item(),
        ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/ccn2/u/haw027/miniconda3/envs/genjax/lib/python3.11/site-packages/warp/types.py", line 2613, in list
    a = self.numpy()
        ^^^^^^^^^^^^
  File "/ccn2/u/haw027/miniconda3/envs/genjax/lib/python3.11/site-packages/warp/types.py", line 2566, in numpy
    a = self.to("cpu", requires_grad=False)
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/ccn2/u/haw027/miniconda3/envs/genjax/lib/python3.11/site-packages/warp/types.py", line 2637, in to
    return warp.clone(self, device=device, requires_grad=requires_grad)
           ^^^^^^^^

Warp CUDA error 700: an illegal memory access was encountered (in function memcpy_d2h, /builds/omniverse/warp/warp/native/warp.cu:743)
Warp CUDA error 700: an illegal memory access was encountered (in function cuda_stream_unregister, /builds/omniverse/warp/warp/native/warp.cu:2444)
E0225 06:42:25.612815  916022 pjrt_stream_executor_client.cc:3045] Execution of replica 0 failed: UNKNOWN: FFI callback error: RuntimeError: Warp copy error: Warp CUDA error 700: an illegal memory access was encountered (in function memcpy_d2h, /builds/omniverse/warp/warp/native/warp.cu:743)


ValueError: UNKNOWN: FFI callback error: RuntimeError: Warp copy error: Warp CUDA error 700: an illegal memory access was encountered (in function memcpy_d2h, /builds/omniverse/warp/warp/native/warp.cu:743)

In [4]:
stepped_model

In [13]:
import warp as wp

a = wp.array([1])
a

In [19]:
a.list()[0]

1

In [12]:
stepped_model["rigid_contact_broad_shape0"].size

1

In [1]:
import warp as wp

In [2]:
a = wp.array([1.0, 2.0, 3.0], dtype=float)
a

Warp 1.7.0.dev20250223 initialized:
   CUDA Toolkit 12.8, Driver 12.2
   Devices:
     "cpu"      : "x86_64"
     "cuda:0"   : "NVIDIA A40" (44 GiB, sm_86, mempool enabled)
     "cuda:1"   : "NVIDIA A40" (47 GiB, sm_86, mempool enabled)
     "cuda:2"   : "NVIDIA A40" (47 GiB, sm_86, mempool enabled)
     "cuda:3"   : "NVIDIA A40" (47 GiB, sm_86, mempool enabled)
     "cuda:4"   : "NVIDIA A40" (47 GiB, sm_86, mempool enabled)
     "cuda:5"   : "NVIDIA A40" (47 GiB, sm_86, mempool enabled)
     "cuda:6"   : "NVIDIA A40" (47 GiB, sm_86, mempool enabled)
     "cuda:7"   : "NVIDIA A40" (47 GiB, sm_86, mempool enabled)
   CUDA peer access:
     Supported fully (all-directional)
   Kernel cache:
     /home/haw027/.cache/warp/1.7.0.dev20250223


In [2]:
wp.vec3(1.0, 2.0, 3.0)

In [4]:
wp.vec3(a[0], a[1], a[2])

RuntimeError: Item indexing is not supported on wp.array objects

In [7]:
wp.array(a, dtype=wp.vec3)

In [1]:
import jax
import jax.numpy as jnp
import warp as wp
from warp.jax_experimental.ffi import jax_callable

@wp.kernel
def scale_kernel(a: wp.array(dtype=float), s: float, output: wp.array(dtype=float)):
    tid = wp.tid()
    output[tid] = a[tid] * s

@wp.kernel
def scale_vec_kernel(a: wp.array(dtype=wp.vec2), s: float, output: wp.array(dtype=wp.vec2)):
    tid = wp.tid()
    output[tid] = a[tid] * s

def example_func(
    # inputs
    a: wp.array(dtype=float),
    b: wp.array(dtype=wp.vec2),
    s: float,
    # outputs
    c: wp.array(dtype=float),
    d: wp.array(dtype=wp.vec2),
    ):
    wp.launch(scale_kernel, dim=a.shape, inputs=[a, s], outputs=[c])
    wp.launch(scale_vec_kernel, dim=b.shape, inputs=[b, s], outputs=[d])

jax_func = jax_callable(example_func, num_outputs=2)

o = wp.array([1, 2, 3])
p = wp.to_jax(o)


Warp 1.7.0.dev20250223 initialized:
   CUDA Toolkit 12.8, Driver 12.2
   Devices:
     "cpu"      : "x86_64"
     "cuda:0"   : "NVIDIA A40" (44 GiB, sm_86, mempool enabled)
     "cuda:1"   : "NVIDIA A40" (47 GiB, sm_86, mempool enabled)
     "cuda:2"   : "NVIDIA A40" (47 GiB, sm_86, mempool enabled)
     "cuda:3"   : "NVIDIA A40" (47 GiB, sm_86, mempool enabled)
     "cuda:4"   : "NVIDIA A40" (47 GiB, sm_86, mempool enabled)
     "cuda:5"   : "NVIDIA A40" (47 GiB, sm_86, mempool enabled)
     "cuda:6"   : "NVIDIA A40" (47 GiB, sm_86, mempool enabled)
     "cuda:7"   : "NVIDIA A40" (47 GiB, sm_86, mempool enabled)
   CUDA peer access:
     Supported fully (all-directional)
   Kernel cache:
     /home/haw027/.cache/warp/1.7.0.dev20250223


ArgumentError: argument 3: TypeError: expected CFunctionType instance instead of CFunctionType

In [2]:
from warp.jax_experimental.ffi import jax_callable
import warp as wp
import jax
import jax.numpy as jnp
import genjax

@wp.kernel
def scale_kernel(a: wp.array(dtype=float), s: float, output: wp.array(dtype=float)):
    tid = wp.tid()
    output[tid] = a[tid] * s

@wp.kernel
def scale_vec_kernel(a: wp.array(dtype=wp.vec2), s: float, output: wp.array(dtype=wp.vec2)):
    tid = wp.tid()
    output[tid] = a[tid] * s


# The Python function to call.
# Note the argument type annotations, just like Warp kernels.
def example_func(
    # inputs
    a: wp.array(dtype=float),
    b: wp.array(dtype=wp.vec2),
    s: float,
    # outputs
    c: wp.array(dtype=float),
    d: wp.array(dtype=wp.vec2),
):
    # launch multiple kernels
    wp.launch(scale_kernel, dim=a.shape, inputs=[a, s], outputs=[c])
    wp.launch(scale_vec_kernel, dim=b.shape, inputs=[b, s], outputs=[d])


jax_func = jax_callable(example_func, num_outputs=2)

# @jax.jit
# def f():
#     # inputs
#     a = jnp.arange(10, dtype=jnp.float32)
#     b = jnp.arange(10, dtype=jnp.float32).reshape((5, 2))  # wp.vec2
#     s = 2.0

#     # output shapes
#     output_dims = {"c": a.shape, "d": b.shape}

#     c, d = jax_func(a, b, s, output_dims=output_dims)

#     return c, d

def make_genjax_func():
    @genjax.gen
    def f():
        # inputs
        a = jnp.arange(10, dtype=jnp.float32)
        b = jnp.arange(10, dtype=jnp.float32).reshape((5, 2))  # wp.vec2
        s = 2.0

        # output shapes
        output_dims = {"c": a.shape, "d": b.shape}

        c, d = jax_func(a, b, s, output_dims=output_dims)

        return c, d
    return f

f = make_genjax_func()
importance_jit = jax.jit(f.importance)
# r1, r2 = f()
# print(r1)
# print(r2)

In [1]:
from warp.jax_experimental.ffi import jax_callable
import warp as wp
import jax.numpy as jnp
import jax
from warp.jax_experimental.ffi import register_ffi_callback
from warp.jax import get_jax_device

In [5]:
@wp.kernel
def scale_kernel(a: wp.array(dtype=float), s: float, output: wp.array(dtype=float)):
    tid = wp.tid()
    output[tid] = a[tid] * s

@wp.kernel
def scale_vec_kernel(a: wp.array(dtype=wp.vec2), s: float, output: wp.array(dtype=wp.vec2)):
    tid = wp.tid()
    output[tid] = a[tid] * s


# The Python function to call.
# Note the argument type annotations, just like Warp kernels.
def example_func(
    # inputs
    a: wp.array(dtype=float),
    b: wp.array(dtype=wp.vec2),
    s: float,
    # outputs
    c: wp.array(dtype=float),
    d: wp.array(dtype=wp.vec2),
):
    # launch multiple kernels
    wp.launch(scale_kernel, dim=a.shape, inputs=[a, s], outputs=[c])
    wp.launch(scale_vec_kernel, dim=b.shape, inputs=[b, s], outputs=[d])


jax_func = jax_callable(example_func, num_outputs=2)

@jax.jit
def f():
    # inputs
    a = jnp.arange(10, dtype=jnp.float32)
    b = jnp.arange(10, dtype=jnp.float32).reshape((5, 2))  # wp.vec2
    s = 2.0

    # output shapes
    # output_dims = {"c": a.shape, "d": b.shape}

    # c, d = jax_func(a, b, s, output_dims=output_dims)
    c, d = jax_func(a, b, s)

    return c, d

r1, r2 = f()
print(r1)
print(r2)

[ 0.  2.  4.  6.  8. 10. 12. 14. 16. 18.]
[[ 0.  2.]
 [ 4.  6.]
 [ 8. 10.]
 [12. 14.]
 [16. 18.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]]


In [7]:
@wp.kernel
def scale_kernel(a: wp.array(dtype=float), s: float, output: wp.array(dtype=float)):
    tid = wp.tid()
    output[tid] = a[tid] * s

@wp.kernel
def scale_twice_kernel(a: wp.array(dtype=float), s: float, output: wp.array(dtype=float)):
    tid = wp.tid()
    output[tid] = a[tid] * s * s

# The Python function to call.
# Note the argument type annotations, just like Warp kernels.
def example_func(
    # inputs
    a: wp.array(dtype=float),
    b: wp.array(dtype=float),
    s: float,
    # outputs
    c: wp.array(dtype=float),
):
    # launch multiple kernels
    wp.launch(scale_kernel, dim=a.shape, inputs=[a, s], outputs=[b])
    wp.launch(scale_twice_kernel, dim=b.shape, inputs=[b, s], outputs=[c])


jax_func = jax_callable(example_func, num_outputs=1)

@jax.jit
def f():
    # inputs
    a = jnp.arange(10, dtype=jnp.float32)
    b = jnp.arange(10, dtype=jnp.float32)
    s = 2.0

    # output shapes
    # output_dims = {"c": a.shape}

    # c = jax_func(a, b, s, output_dims=output_dims)
    c = jax_func(a, b, s)

    return c

r1 = f()
print(r1)

[Array([ 0.,  8., 16., 24., 32., 40., 48., 56., 64., 72.], dtype=float32)]


In [ ]:
from warp.sim.integrator import integrate_bodies

In [ ]:
@wp.kernel
def eval_rigid_contacts(
    body_q: wp.array(dtype=wp.transform),
    body_qd: wp.array(dtype=wp.spatial_vector),
    body_com: wp.array(dtype=wp.vec3),
    ke: wp.array(dtype=float),
    kd: wp.array(dtype=float),
    kf: wp.array(dtype=float),
    ka: wp.array(dtype=float),
    mu: wp.array(dtype=float),
    shape_body: wp.array(dtype=int),
    contact_count: wp.array(dtype=int),
    contact_point0: wp.array(dtype=wp.vec3),
    contact_point1: wp.array(dtype=wp.vec3),
    contact_normal: wp.array(dtype=wp.vec3),
    contact_shape0: wp.array(dtype=int),
    contact_shape1: wp.array(dtype=int),
    force_in_world_frame: bool,
    friction_smoothing: float,
    # outputs
    body_f: wp.array(dtype=wp.spatial_vector),
):
    tid = wp.tid()

    count = contact_count[0]
    if tid >= count:
        return

    # retrieve contact thickness, compute average contact material properties
    ke = 0.0  # contact normal force stiffness
    kd = 0.0  # damping coefficient
    kf = 0.0  # friction force stiffness
    ka = 0.0  # adhesion distance
    mu = 0.0  # friction coefficient
    mat_nonzero = 0
    thickness_a = 0.0
    thickness_b = 0.0
    shape_a = contact_shape0[tid]
    shape_b = contact_shape1[tid]
    if shape_a == shape_b:
        return
    body_a = -1
    body_b = -1
    if shape_a >= 0:
        mat_nonzero += 1
        ke += ke[shape_a]
        kd += kd[shape_a]
        kf += kf[shape_a]
        ka += ka[shape_a]
        mu += mu[shape_a]
        thickness_a = 1.e-05
        body_a = shape_body[shape_a]
    if shape_b >= 0:
        mat_nonzero += 1
        ke += ke[shape_b]
        kd += kd[shape_b]
        kf += kf[shape_b]
        ka += ka[shape_b]
        mu += mu[shape_b]
        thickness_b = 1.e-05
        body_b = shape_body[shape_b]
    if mat_nonzero > 0:
        ke /= float(mat_nonzero)
        kd /= float(mat_nonzero)
        kf /= float(mat_nonzero)
        ka /= float(mat_nonzero)
        mu /= float(mat_nonzero)

    # contact normal in world space
    n = contact_normal[tid]
    bx_a = contact_point0[tid]
    bx_b = contact_point1[tid]
    r_a = wp.vec3(0.0)
    r_b = wp.vec3(0.0)
    if body_a >= 0:
        X_wb_a = body_q[body_a]
        X_com_a = body_com[body_a]
        bx_a = wp.transform_point(X_wb_a, bx_a) - thickness_a * n
        r_a = bx_a - wp.transform_point(X_wb_a, X_com_a)

    if body_b >= 0:
        X_wb_b = body_q[body_b]
        X_com_b = body_com[body_b]
        bx_b = wp.transform_point(X_wb_b, bx_b) + thickness_b * n
        r_b = bx_b - wp.transform_point(X_wb_b, X_com_b)

    d = wp.dot(n, bx_a - bx_b)

    if d >= ka:
        return

    # compute contact point velocity
    bv_a = wp.vec3(0.0)
    bv_b = wp.vec3(0.0)
    if body_a >= 0:
        body_v_s_a = body_qd[body_a]
        body_w_a = wp.spatial_top(body_v_s_a)
        body_v_a = wp.spatial_bottom(body_v_s_a)
        if force_in_world_frame:
            bv_a = body_v_a + wp.cross(body_w_a, bx_a)
        else:
            bv_a = body_v_a + wp.cross(body_w_a, r_a)

    if body_b >= 0:
        body_v_s_b = body_qd[body_b]
        body_w_b = wp.spatial_top(body_v_s_b)
        body_v_b = wp.spatial_bottom(body_v_s_b)
        if force_in_world_frame:
            bv_b = body_v_b + wp.cross(body_w_b, bx_b)
        else:
            bv_b = body_v_b + wp.cross(body_w_b, r_b)

    # relative velocity
    v = bv_a - bv_b

    # print(v)

    # decompose relative velocity
    vn = wp.dot(n, v)
    vt = v - n * vn

    # contact elastic
    fn = d * ke

    # contact damping
    fd = wp.min(vn, 0.0) * kd * wp.step(d)

    # viscous friction
    # ft = vt*kf

    # Coulomb friction (box)
    # lower = mu * d * ke
    # upper = -lower

    # vx = wp.clamp(wp.dot(wp.vec3(kf, 0.0, 0.0), vt), lower, upper)
    # vz = wp.clamp(wp.dot(wp.vec3(0.0, 0.0, kf), vt), lower, upper)

    # ft = wp.vec3(vx, 0.0, vz)

    # Coulomb friction (smooth, but gradients are numerically unstable around |vt| = 0)
    ft = wp.vec3(0.0)
    if d < 0.0:
        # use a smooth vector norm to avoid gradient instability at/around zero velocity
        vs = wp.norm_huber(vt, delta=friction_smoothing)
        if vs > 0.0:
            fr = vt / vs
            ft = fr * wp.min(kf * vs, -mu * (fn + fd))

    f_total = n * (fn + fd) + ft
    # f_total = n * (fn + fd)
    # f_total = n * fn

    if body_a >= 0:
        if force_in_world_frame:
            wp.atomic_add(body_f, body_a, wp.spatial_vector(wp.cross(bx_a, f_total), f_total))
        else:
            wp.atomic_sub(body_f, body_a, wp.spatial_vector(wp.cross(r_a, f_total), f_total))

    if body_b >= 0:
        if force_in_world_frame:
            wp.atomic_sub(body_f, body_b, wp.spatial_vector(wp.cross(bx_b, f_total), f_total))
        else:
            wp.atomic_add(body_f, body_b, wp.spatial_vector(wp.cross(r_b, f_total), f_total))


In [ ]:
def simulate(
    # inputs
    rigid_contact_max: int,
    body_q: wp.array(dtype=wp.transform),
    body_qd: wp.array(dtype=wp.spatial_vector),
    body_com: wp.array(dtype=wp.vec3),
    ke: wp.array(dtype=float),
    kd: wp.array(dtype=float),
    kf: wp.array(dtype=float),
    ka: wp.array(dtype=float),
    mu: wp.array(dtype=float),
    shape_body: wp.array(dtype=int),
    rigid_contact_count: wp.array(dtype=int),
    rigid_contact_point0: wp.array(dtype=wp.vec3),
    rigid_contact_point1: wp.array(dtype=wp.vec3),
    rigid_contact_normal: wp.array(dtype=wp.vec3),
    rigid_contact_shape0: wp.array(dtype=int),
    rigid_contact_shape1: wp.array(dtype=int),
    force_in_world_frame: bool = False,
    friction_smoothing: float = 1.0,
    # outputs
    body_f: wp.array(dtype=wp.spatial_vector),

    # inputs
    body_mass: wp.array(dtype=float),
    body_inertia: wp.array(dtype=wp.mat33),
    body_inv_mass: wp.array(dtype=float),
    body_inv_inertia: wp.array(dtype=wp.mat33),
    gravity: wp.vec3,
    angular_damping: float = 0.05,
    dt: float,
    body_count: int,
    # outputs
    body_q_new: wp.array(dtype=wp.transform),
    body_qd_new: wp.array(dtype=wp.spatial_vector),
):
    # collision
    if model.shape_contact_pair_count or model.ground and model.shape_ground_contact_pair_count:
        # clear old count
        model.rigid_contact_count.zero_()

        model.rigid_contact_broad_shape0.fill_(-1)
        model.rigid_contact_broad_shape1.fill_(-1)

    if model.shape_contact_pair_count:
        wp.launch(
            kernel=broadphase_collision_pairs,
            dim=model.shape_contact_pair_count,
            inputs=[
                model.shape_contact_pairs,
                state.body_q,
                model.shape_transform,
                model.shape_body,
                model.body_mass,
                model.shape_count,
                model.shape_geo,
                model.shape_collision_radius,
                model.rigid_contact_max,
                model.rigid_contact_margin,
                model.rigid_mesh_contact_max,
                iterate_mesh_vertices,
            ],
            outputs=[
                model.rigid_contact_count,
                model.rigid_contact_broad_shape0,
                model.rigid_contact_broad_shape1,
                model.rigid_contact_point_id,
                model.rigid_contact_point_limit,
            ],
            record_tape=False,
        )

    if model.ground and model.shape_ground_contact_pair_count:
        wp.launch(
            kernel=broadphase_collision_pairs,
            dim=model.shape_ground_contact_pair_count,
            inputs=[
                model.shape_ground_contact_pairs,
                state.body_q,
                model.shape_transform,
                model.shape_body,
                model.body_mass,
                model.shape_count,
                model.shape_geo,
                model.shape_collision_radius,
                model.rigid_contact_max,
                model.rigid_contact_margin,
                model.rigid_mesh_contact_max,
                iterate_mesh_vertices,
            ],
            outputs=[
                model.rigid_contact_count,
                model.rigid_contact_broad_shape0,
                model.rigid_contact_broad_shape1,
                model.rigid_contact_point_id,
                model.rigid_contact_point_limit,
            ],
            record_tape=False,
        )

    if model.shape_contact_pair_count or model.ground and model.shape_ground_contact_pair_count:
        model.rigid_contact_count.zero_()
        model.rigid_contact_tids.zero_()
        if model.rigid_contact_pairwise_counter is not None:
            model.rigid_contact_pairwise_counter.zero_()
        model.rigid_contact_shape0.fill_(-1)
        model.rigid_contact_shape1.fill_(-1)

        wp.launch(
            kernel=handle_contact_pairs,
            dim=model.rigid_contact_max,
            inputs=[
                state.body_q,
                model.shape_transform,
                model.shape_body,
                model.shape_geo,
                model.rigid_contact_margin,
                model.rigid_contact_broad_shape0,
                model.rigid_contact_broad_shape1,
                model.shape_count,
                model.rigid_contact_point_id,
                model.rigid_contact_point_limit,
                edge_sdf_iter,
            ],
            outputs=[
                model.rigid_contact_count,
                model.rigid_contact_shape0,
                model.rigid_contact_shape1,
                model.rigid_contact_point0,
                model.rigid_contact_point1,
                model.rigid_contact_offset0,
                model.rigid_contact_offset1,
                model.rigid_contact_normal,
                model.rigid_contact_thickness,
                model.rigid_contact_pairwise_counter,
                model.rigid_contact_tids,
            ],
        )

    # compute forces
    wp.launch(
        kernel=eval_rigid_contacts,
        dim=rigid_contact_max,
        inputs=[
            body_q,
            body_qd,
            body_com,
            ke,
            kd,
            kf,
            ka,
            mu,
            shape_body,
            rigid_contact_count,
            rigid_contact_point0,
            rigid_contact_point1,
            rigid_contact_normal,
            rigid_contact_shape0,
            rigid_contact_shape1,
            force_in_world_frame,
            friction_smoothing,
        ],
        outputs=[body_f],
    )
    
    # integrate
    wp.launch(
        kernel=integrate_bodies,
        dim=body_count,
        inputs=[
            body_q,
            body_qd,
            body_f,
            body_com,
            body_mass,
            body_inertia,
            body_inv_mass,
            body_inv_inertia,
            gravity,
            angular_damping,
            dt,
        ],
        outputs=[body_q_new, body_qd_new],
    )

# test physion

In [1]:
import numpy as np
import trimesh
import warp as wp
import warp.sim.render
from os.path import isfile, join
from os import listdir
import h5py
import jax

In [2]:
def euler_angles_to_quaternion(euler: np.ndarray) -> np.ndarray:
    """
    Convert Euler angles to a quaternion.

    Source: https://pastebin.com/riRLRvch

    :param euler: The Euler angles vector.

    :return: The quaternion representation of the Euler angles.
    """
    pitch = np.radians(euler[0] * 0.5)
    cp = np.cos(pitch)
    sp = np.sin(pitch)

    yaw = np.radians(euler[1] * 0.5)
    cy = np.cos(yaw)
    sy = np.sin(yaw)

    roll = np.radians(euler[2] * 0.5)
    cr = np.cos(roll)
    sr = np.sin(roll)

    x = sy * cp * sr + cy * sp * cr
    y = sy * cp * cr - cy * sp * sr
    z = cy * cp * sr - sy * sp * cr
    w = cy * cp * cr + sy * sp * sr
    return np.array([x, y, z, w])

class Example:
    def __init__(self, stage_path="example_rigid_contact.usd", hdf5_path=''):
        builder = wp.sim.ModelBuilder()

        self.sim_time = 0.0
        fps = 100
        self.frame_dt = 1.0 / fps

        self.sim_substeps = 10
        self.sim_dt = self.frame_dt / self.sim_substeps

        self.mu = 0.25
        self.restitution = 0.4

        self.mesh_path = '/ccn2/u/rmvenkat/data/all_flex_meshes'
        self.mesh_lib = dict([(f, join(self.mesh_path, f)) for f in listdir(self.mesh_path) if isfile(join(self.mesh_path, f)) and join(self.mesh_path, f).endswith('.obj')])
        initial_positions, initial_rotations, model_names, scales = self.load_hdf5(hdf5_path)

        # meshes
        for i, (pos, rot, name, scale) in enumerate(zip(initial_positions, initial_rotations, model_names, scales)):
            b = builder.add_body(
                origin=wp.transform(
                    pos, euler_angles_to_quaternion(rot),
                )
            )

            mesh = self.load_mesh(self.mesh_lib[name.decode('utf-8')+'.obj'])
            builder.add_shape_mesh(
                body=b,
                mesh=mesh,
                pos=wp.vec3(0.0, 0.0, 0.0),
                scale=scale,
                # density=5,
                restitution=self.restitution,
                mu=self.mu,
                # ke=self.ke,
                # kd=self.kd,
                # kf=self.kf,
                density=1e3,
                has_ground_collision=True,
                has_shape_collision=True
            )
        builder.set_ground_plane(mu=self.mu)
        
        # finalize model
        self.model = builder.finalize()
        self.model.ground = True
        print(f"self.model.rigid_contact_count: {self.model.rigid_contact_count} {type(self.model.rigid_contact_count)}")
        print(f"to jax: {wp.to_jax(self.model.rigid_contact_count)} {type(wp.to_jax(self.model.rigid_contact_count))}")

        print(f"self.model.rigid_mesh_contact_max: {self.model.rigid_mesh_contact_max},\n self.model.ground: {self.model.ground},\n self.model.shape_materials: {self.model.shape_materials},\n self.model.shape_geo: {self.model.shape_geo},\n self.model.device: {self.model.device} {type(self.model.device)},\n self.model.spring_count: {self.model.spring_count},\n self.model.tri_count: {self.model.tri_count},\n self.model.enable_tri_collisions: {self.model.enable_tri_collisions},\n self.model.edge_count: {self.model.edge_count},\n self.model.particle_count: {self.model.particle_count},\n self.model.tet_count: {self.model.tet_count},\n self.model.rigid_contact_max: {self.model.rigid_contact_max},\n self.model.ground: {self.model.ground},\n self.model.shape_ground_contact_pair_count: {self.model.shape_ground_contact_pair_count},\n self.model.shape_contact_pair_count: {self.model.shape_contact_pair_count},\n self.model.joint_count: {self.model.joint_count},\n self.model.particle_count: {self.model.particle_count},\n self.model.shape_count: {self.model.shape_count},\n self.model.muscle_count: {self.model.muscle_count}")

        

        self.integrator = wp.sim.SemiImplicitIntegrator()

        if stage_path:
            self.renderer = wp.sim.render.SimRenderer(self.model, stage_path, scaling=0.5)
        else:
            self.renderer = None

        self.state_0 = self.model.state()
        self.state_1 = self.model.state()

        wp.sim.eval_fk(self.model, self.model.joint_q, self.model.joint_qd, None, self.state_0)

        self.use_cuda_graph = wp.get_device().is_cuda
        if self.use_cuda_graph:
            with wp.ScopedCapture() as capture:
                self.simulate()
            self.graph = capture.graph

    def load_hdf5(self, path):
        with h5py.File(path, "r") as f:
            distractors = (
                np.array(f["static"]["distractors"])
                if np.array(f["static"]["distractors"]).size != 0
                else []
            )
            occluders = (
                np.array(f["static"]["occluders"])
                if np.array(f["static"]["occluders"]).size != 0
                else []
            )
            distractors_occluders = np.concatenate([distractors, occluders])

            model_names = np.array(f["static"]["model_names"])[1:]

            scales = np.array(
                f["static"]["scale"]
            )[1:]
            initial_positions = np.array(f["static"]["initial_position"])[1:]
            initial_rotations = np.array(f["static"]["initial_rotation"])[1:]
            
            if len(distractors_occluders):
                model_names = model_names[: -len(distractors_occluders)]
                scales = scales[
                    : -len(distractors_occluders)
                ]
                initial_positions = initial_positions[
                    : -len(distractors_occluders)
                ]
                initial_rotations = initial_rotations[
                    : -len(distractors_occluders)
                ]
        return initial_positions, initial_rotations, model_names, scales

    def load_mesh(self, path):
        m = trimesh.load(path)
        mesh_points = np.array(m.vertices)
        mesh_indices = np.array(m.faces, dtype=np.int32).flatten()
        mesh = wp.sim.Mesh(mesh_points, mesh_indices)
        return mesh

    def simulate(self):
        for _ in range(self.sim_substeps):
            self.state_0.clear_forces()
            wp.sim.collide(self.model, self.state_0)
            self.integrator.simulate(self.model, self.state_0, self.state_1, self.sim_dt)
            self.state_0, self.state_1 = self.state_1, self.state_0
            
    def step(self):
        with wp.ScopedTimer("step", active=True):
            if self.use_cuda_graph:
                wp.capture_launch(self.graph)
                print(self.state_0.body_q, self.state_1.body_q)
            # else:
            #     self.simulate()
        self.sim_time += self.frame_dt

    def render(self):
        if self.renderer is None:
            return

        with wp.ScopedTimer("render", active=True):
            self.renderer.begin_frame(self.sim_time)
            self.renderer.render(self.state_0)
            self.renderer.end_frame()

In [ ]:
if __name__ == "__main__":
    import argparse

    parser = argparse.ArgumentParser(formatter_class=argparse.ArgumentDefaultsHelpFormatter)
    parser.add_argument("--device", type=str, default=None, help="Override the default Warp device.")
    parser.add_argument(
        "--stage_path",
        type=lambda x: None if x == "None" else str(x),
        default="example_rigid_contact.usd",
        help="Path to the output USD file.",
    )
    parser.add_argument(
        "--hdf5_path",
        type=lambda x: None if x == "None" else str(x),
        default="/ccn2/u/rmvenkat/data/testing_physion/regenerate_from_old_commit/test_humans_consolidated/lf_0/support_all_movies/pilot_towers_nb3_fr015_SJ025_mono1_dis0_occ0_tdwroom_unstable_0004.hdf5",
        help="Path to the input hdf5 file.",
    )
    parser.add_argument("--num_frames", type=int, default=250, help="Total number of frames.")
    args = parser.parse_known_args()[0]

    with wp.ScopedDevice(args.device):
        example = Example(stage_path=args.stage_path, hdf5_path=args.hdf5_path)

        for _ in range(args.num_frames):
            example.step()
            example.render()

        if example.renderer:
            example.renderer.save()

Warp 1.7.0.dev20250223 initialized:
   CUDA Toolkit 12.8, Driver 12.2
   Devices:
     "cpu"      : "x86_64"
     "cuda:0"   : "NVIDIA A40" (44 GiB, sm_86, mempool enabled)
     "cuda:1"   : "NVIDIA A40" (47 GiB, sm_86, mempool enabled)
     "cuda:2"   : "NVIDIA A40" (47 GiB, sm_86, mempool enabled)
     "cuda:3"   : "NVIDIA A40" (47 GiB, sm_86, mempool enabled)
     "cuda:4"   : "NVIDIA A40" (47 GiB, sm_86, mempool enabled)
     "cuda:5"   : "NVIDIA A40" (47 GiB, sm_86, mempool enabled)
     "cuda:6"   : "NVIDIA A40" (47 GiB, sm_86, mempool enabled)
     "cuda:7"   : "NVIDIA A40" (47 GiB, sm_86, mempool enabled)
   CUDA peer access:
     Supported fully (all-directional)
   Kernel cache:
     /home/haw027/.cache/warp/1.7.0.dev20250223
Module warp.sim.inertia 185ee85 load on device 'cuda:0' took 1.05 ms  (cached)
Module warp.sim.collide 26674e6 load on device 'cuda:0' took 1.29 ms  (cached)
self.model.rigid_contact_count: [0] <class 'warp.types.array'>
to jax: [0] <class 'jaxlib.xla_ex

ImportError: Failed to import pxr. Please install USD (e.g. via `pip install usd-core`).

: 